In [ ]:
!pip install vtk numpy tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 MB 7.4 MB/s eta 0:00:00


In [ ]:
import vtk
import numpy as np

def load_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()

    image_data = reader.GetOutput()
    dims = image_data.GetDimensions()  # (nx, ny, nz)

    # Extract scalar arrays
    point_data = image_data.GetPointData()
    mean_array = point_data.GetArray("Pressure_Mean")
    std_array = point_data.GetArray("Pressure_Std")

    n_points = mean_array.GetNumberOfTuples()
    mean_np = np.array([mean_array.GetValue(i) for i in range(n_points)])
    std_np = np.array([std_array.GetValue(i) for i in range(n_points)])

    # Reshape to 3D
    mean_3d = mean_np.reshape(dims, order="F")
    std_3d = std_np.reshape(dims, order="F")

    return mean_3d, std_3d, dims


In [ ]:
from tqdm import tqdm

def estimate_crossing_prob(mu_vals, sigma_vals, isovalue, num_samples=1000):
    samples = np.random.randn(num_samples, 8) * sigma_vals + mu_vals
    above = np.any(samples > isovalue, axis=1)
    below = np.any(samples < isovalue, axis=1)
    crossing = above & below
    return np.mean(crossing)

def probabilistic_marching_cubes(mean_3d, std_3d, isovalue, num_samples=1000):
    nx, ny, nz = mean_3d.shape
    prob_vol = np.zeros((nx - 1, ny - 1, nz - 1))

    for i in tqdm(range(nx - 1), desc="PMC"):
        for j in range(ny - 1):
            for k in range(nz - 1):
                corners = [(i+di, j+dj, k+dk) for di in [0, 1] for dj in [0, 1] for dk in [0, 1]]
                mu_vals = np.array([mean_3d[x, y, z] for x, y, z in corners])
                sigma_vals = np.array([std_3d[x, y, z] for x, y, z in corners])
                prob = estimate_crossing_prob(mu_vals, sigma_vals, isovalue, num_samples)
                prob_vol[i, j, k] = prob

    return prob_vol


In [ ]:
def save_prob_vti(prob_vol, output_filename="probability.vti"):
    nx, ny, nz = prob_vol.shape
    image_data = vtk.vtkImageData()
    image_data.SetDimensions(nx, ny, nz)
    image_data.SetSpacing(1.0, 1.0, 1.0)
    image_data.SetOrigin(0, 0, 0)

    # Convert to VTK array
    flat = prob_vol.flatten(order="F")
    vtk_array = vtk.vtkFloatArray()
    vtk_array.SetName("probability")
    vtk_array.SetNumberOfValues(flat.size)
    for i, val in enumerate(flat):
        vtk_array.SetValue(i, val)

    image_data.GetPointData().AddArray(vtk_array)

    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(output_filename)
    writer.SetInputData(image_data)
    writer.Write()


In [ ]:
mean_3d, std_3d, dims = load_vti("isabel_gaussian.vti")
isovalue = 930.0

prob_vol = probabilistic_marching_cubes(mean_3d, std_3d, isovalue, num_samples=2000)
save_prob_vti(prob_vol, "isabel_probability.vti")


PMC: 100%|██████████| 249/249 [37:28<00:00,  9.03s/it]


In [ ]:
import vtk

def print_vti_arrays(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()

    image_data = reader.GetOutput()
    point_data = image_data.GetPointData()

    print("Available scalar arrays:")
    for i in range(point_data.GetNumberOfArrays()):
        print("-", point_data.GetArrayName(i))

print_vti_arrays("/content/isabel_gaussian.vti")


Available scalar arrays:
- Pressure_Mean
- Pressure_Std


#gaussian

In [ ]:
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from tqdm import tqdm

# === CONFIG ===
INPUT_VTI = "/content/Teardrop_Gaussian.vti"
MEAN_NAME = "mean"
VAR_NAME = "variance"
ISOLEVEL =  159.9798
N_SAMPLES = 1000
OUTPUT_VTI = "isosurface_probabilities_320.vti"
CORRELATION = 0# Synthetic correlation coefficient for adjacent cell vertices

# === Load VTI File ===
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(INPUT_VTI)
reader.Update()
image = reader.GetOutput()
dims = image.GetDimensions()

mean_arr = vtk_to_numpy(image.GetPointData().GetArray(0))
var_arr= vtk_to_numpy(image.GetPointData().GetArray(1))

mean_3d = mean_arr.reshape(dims[::-1], order='F')
std_3d = var_arr.reshape(dims[::-1], order='F')

# === Cell Vertex Indexing ===
def get_cell_vertices(i, j, k):
    return [
        (i,   j,   k),
        (i+1, j,   k),
        (i+1, j+1, k),
        (i,   j+1, k),
        (i,   j,   k+1),
        (i+1, j,   k+1),
        (i+1, j+1, k+1),
        (i,   j+1, k+1),
    ]

# === Build synthetic 8x8 covariance matrix ===
def build_covariance(sigma, rho=0.6):
    cov = np.full((8, 8), rho)
    np.fill_diagonal(cov, 1.0)
    return (sigma[:, None] * sigma[None, :]) * cov

# === Check if isosurface crosses the cell ===
def is_crossing(values, isovalue):
    return np.min(values) < isovalue and np.max(values) > isovalue

# === Monte Carlo Estimation (with correlation) ===
def sample_crossing_probability(mu, sigma, isovalue, n_samples, rho):
    cov = build_covariance(sigma, rho)
    try:
        samples = np.random.multivariate_normal(mu, cov, size=n_samples)
    except np.linalg.LinAlgError:
        print("hi")
        # fallback in case of numerical issues
        samples = np.random.normal(loc=mu, scale=sigma, size=(n_samples, 8))
    cross = np.any(samples < isovalue, axis=1) & np.any(samples > isovalue, axis=1)
    return np.mean(cross)

# === Main Computation ===
output_shape = tuple(d - 1 for d in mean_3d.shape)
prob_grid = np.zeros(output_shape, dtype=np.float32)

total_cells = np.prod(output_shape)
progress = tqdm(total=total_cells, desc="Processing Cells")

print("Computing probabilities with correlation...")
for i in range(output_shape[0]):
    for j in range(output_shape[1]):
        for k in range(output_shape[2]):
            verts = get_cell_vertices(i, j, k)
            mu = np.array([mean_3d[x, y, z] for (x, y, z) in verts], dtype=np.float64)
            sigma = np.array([std_3d[x, y, z] for (x, y, z) in verts], dtype=np.float64)
            prob = sample_crossing_probability(mu, sigma, ISOLEVEL, N_SAMPLES, CORRELATION)
            prob_grid[i, j, k] = prob
            progress.update(1)

progress.close()

# === Convert to VTK ImageData ===
prob_image = vtk.vtkImageData()
prob_image.SetDimensions(prob_grid.shape)
prob_image.SetSpacing(image.GetSpacing())
prob_image.SetOrigin(image.GetOrigin())

flat = prob_grid.ravel(order='F')  # VTK expects Fortran order
vtk_array = numpy_to_vtk(flat, deep=True)
vtk_array.SetName("crossing_probability")
prob_image.GetPointData().AddArray(vtk_array)
prob_image.GetPointData().SetActiveScalars("crossing_probability")

# === Write Output ===
writer = vtk.vtkXMLImageDataWriter()
writer.SetFileName(OUTPUT_VTI)
writer.SetInputData(prob_image)
writer.Write()

print(f"Saved output to {OUTPUT_VTI}")


Processing Cells:   0%|          | 16/250047 [00:00<26:24, 157.83it/s]

Computing probabilities with correlation...


Processing Cells:   1%|          | 2057/250047 [00:15<26:34, 155.48it/s]

KeyboardInterrupt: 

#adapative

In [ ]:
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from tqdm import tqdm

# === CONFIG ===
INPUT_VTI = "/content/new_gmm_teardrop_ema_320_3 _layers_week_8_predict.vti"
MEAN_NAME = "mean"
VAR_NAME = "variance"
ISOLEVEL = 159.9798
OUTPUT_VTI = "adaptive_isosurface_probabilities_320.vti"
CORRELATION = 0  # Correlation between cell vertices

# Adaptive Sampling Parameters
MIN_SAMPLES = 200
MAX_SAMPLES = 10000

# === Load VTI File ===
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(INPUT_VTI)
reader.Update()
image = reader.GetOutput()
dims = image.GetDimensions()

mean_arr = vtk_to_numpy(image.GetPointData().GetArray(0))
var_arr = vtk_to_numpy(image.GetPointData().GetArray(1))

mean_3d = mean_arr.reshape(dims[::-1], order='F')
std_3d = var_arr.reshape(dims[::-1], order='F')

# === Cell Vertex Indexing ===
def get_cell_vertices(i, j, k):
    return [
        (i,   j,   k),
        (i+1, j,   k),
        (i+1, j+1, k),
        (i,   j+1, k),
        (i,   j,   k+1),
        (i+1, j,   k+1),
        (i+1, j+1, k+1),
        (i,   j+1, k+1),
    ]

# === Build synthetic 8x8 covariance matrix ===
def build_covariance(sigma, rho=0.6):
    cov = np.full((8, 8), rho)
    np.fill_diagonal(cov, 1.0)
    return (sigma[:, None] * sigma[None, :]) * cov

# === Check if isosurface crosses the cell ===
def is_crossing(values, isovalue):
    return np.min(values) < isovalue and np.max(values) > isovalue

# === Monte Carlo Estimation (with correlation) ===
def sample_crossing_probability(mu, sigma, isovalue, n_samples, rho):
    cov = build_covariance(sigma, rho)
    try:
        samples = np.random.multivariate_normal(mu, cov, size=n_samples)
    except np.linalg.LinAlgError:
        samples = np.random.normal(loc=mu, scale=sigma, size=(n_samples, 8))
    cross = np.any(samples < isovalue, axis=1) & np.any(samples > isovalue, axis=1)
    return np.mean(cross)

# === Adaptive Sample Count Function ===
def get_adaptive_sample_count(avg_distance, min_samples=200, max_samples=5000):
    weight = np.exp(-avg_distance / 20)  # adjust denominator to tune sharpness
    return int(min_samples + (max_samples - min_samples) * weight)

# === Main Computation ===
output_shape = tuple(d - 1 for d in mean_3d.shape)
prob_grid = np.zeros(output_shape, dtype=np.float32)

total_cells = np.prod(output_shape)
progress = tqdm(total=total_cells, desc="Processing Cells")

print("Computing adaptive probabilities with correlation...")
for i in range(output_shape[0]):
    for j in range(output_shape[1]):
        for k in range(output_shape[2]):
            verts = get_cell_vertices(i, j, k)
            mu = np.array([mean_3d[x, y, z] for (x, y, z) in verts], dtype=np.float64)
            sigma = np.array([std_3d[x, y, z] for (x, y, z) in verts], dtype=np.float64)
            avg_distance = np.mean(np.abs(mu - ISOLEVEL))
            n_samples = get_adaptive_sample_count(avg_distance, MIN_SAMPLES, MAX_SAMPLES)
            prob = sample_crossing_probability(mu, sigma, ISOLEVEL, n_samples, CORRELATION)
            prob_grid[i, j, k] = prob
            progress.update(1)

progress.close()

# === Convert to VTK ImageData ===
prob_image = vtk.vtkImageData()
prob_image.SetDimensions(prob_grid.shape)
prob_image.SetSpacing(image.GetSpacing())
prob_image.SetOrigin(image.GetOrigin())

flat = prob_grid.ravel(order='F')  # VTK expects Fortran order
vtk_array = numpy_to_vtk(flat, deep=True)
vtk_array.SetName("crossing_probability")
prob_image.GetPointData().AddArray(vtk_array)
prob_image.GetPointData().SetActiveScalars("crossing_probability")

# === Write Output ===
writer = vtk.vtkXMLImageDataWriter()
writer.SetFileName(OUTPUT_VTI)
writer.SetInputData(prob_image)
writer.Write()

print(f"Saved output to {OUTPUT_VTI}")


Processing Cells:   0%|          | 265/250047 [00:00<01:34, 2649.14it/s]

Computing adaptive probabilities with correlation...


Processing Cells: 100%|██████████| 250047/250047 [07:47<00:00, 535.03it/s] 

Saved output to adaptive_isosurface_probabilities_320.vti


#gmm

In [ ]:
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from tqdm import tqdm

# === CONFIG ===
INPUT_VTI = "data\Isabel_3D_gmm.vti"
OUTPUT_VTI = "isosurface_probabilities_8D_GMM_my_one.vti"
ISOLEVEL = 159.9798
N_SAMPLES = 1000

# === Load VTI File ===
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(INPUT_VTI)
reader.Update()
image = reader.GetOutput()
dims = image.GetDimensions()

# === Load GMM Arrays (Means, Stds, Weights) ===
means = [vtk_to_numpy(image.GetPointData().GetArray(f"GMM_Mean{i}")) for i in range(3)]
stds = [vtk_to_numpy(image.GetPointData().GetArray(f"GMM_Std{i}")) for i in range(3)]
weights = [vtk_to_numpy(image.GetPointData().GetArray(f"GMM_Weight{i}")) for i in range(3)]
print(len(means),len(means[0]),len(stds),len(stds[0]),len(weights),len(weights[0]))
# === Reshape Arrays to 3D Grids ===
gmm_means_3d = [arr.reshape(dims[::-1], order='F') for arr in means]
gmm_stds_3d = [arr.reshape(dims[::-1], order='F') for arr in stds]
gmm_weights_3d = [arr.reshape(dims[::-1], order='F') for arr in weights]
print(len(gmm_means_3d),len(gmm_means_3d[0]),len(gmm_stds_3d),len(gmm_stds_3d[0]),len(gmm_weights_3d),len(gmm_weights_3d[0]))
for i, mean in enumerate(gmm_means_3d):
    print(f"Component {i} shape: {mean.shape}")
# Shape of the 3D grid for component 0

# === Cell Vertex Indexing ===
def get_cell_vertices(i, j, k):
    return [
        (i,   j,   k),
        (i+1, j,   k),
        (i+1, j+1, k),
        (i,   j+1, k),
        (i,   j,   k+1),
        (i+1, j,   k+1),
        (i+1, j+1, k+1),
        (i,   j+1, k+1),
    ]

# === GMM Component Sampler ===
def sample_8d_gmm(weights, means, covs, n_samples):
    comps = np.random.choice(3, size=n_samples, p=weights)
    samples = np.empty((n_samples, 8))
    for c in range(3):
        idx = np.where(comps == c)[0]
        if len(idx) > 0:
            samples[idx] = np.random.multivariate_normal(mean=means[c], cov=covs[c], size=len(idx))
    return samples

# === Probability Estimator ===
def sample_crossing_probability_joint_gmm(weights, means, covs, isovalue, n_samples):
    samples = sample_8d_gmm(weights, means, covs, n_samples)
    cross = np.any(samples < isovalue, axis=1) & np.any(samples > isovalue, axis=1)
    return np.mean(cross)

# === Main Computation ===
output_shape = tuple(d - 1 for d in dims[::-1])
prob_grid = np.zeros(output_shape, dtype=np.float32)

progress = tqdm(total=np.prod(output_shape), desc="Processing 8D GMM Cells")

for i in range(output_shape[0]):
    for j in range(output_shape[1]):
        for k in range(output_shape[2]):
            verts = get_cell_vertices(i, j, k)

            # For each component (0,1,2), build mean/cov across 8 vertices
            means_list = []
            covs_list = []
            weights_raw = []

            for c in range(3):
                mu_c = [gmm_means_3d[c][x, y, z] for (x, y, z) in verts]
                std_c = [gmm_stds_3d[c][x, y, z] for (x, y, z) in verts]
                cov_c = np.diag(np.square(std_c))

                means_list.append(np.array(mu_c))
                covs_list.append(cov_c)

                w_c = [gmm_weights_3d[c][x, y, z] for (x, y, z) in verts]
                w_product = np.prod(w_c)  # product of weights across 8 corners
                weights_raw.append(w_product)

            # Normalize weights
            weights_array = np.array(weights_raw)
            weights_array /= np.sum(weights_array)

            # Estimate probability
            prob = sample_crossing_probability_joint_gmm(weights_array, means_list, covs_list, ISOLEVEL, N_SAMPLES)
            prob_grid[i, j, k] = prob
            progress.update(1)

progress.close()

# === Convert to VTK ImageData ===
prob_image = vtk.vtkImageData()
prob_image.SetDimensions(prob_grid.shape)
prob_image.SetSpacing(image.GetSpacing())
prob_image.SetOrigin(image.GetOrigin())

flat = prob_grid.ravel(order='F')  # Fortran order
vtk_array = numpy_to_vtk(flat, deep=True)
vtk_array.SetName("crossing_probability")
prob_image.GetPointData().AddArray(vtk_array)
prob_image.GetPointData().SetActiveScalars("crossing_probability")

# === Write Output ===
writer = vtk.vtkXMLImageDataWriter()
writer.SetFileName(OUTPUT_VTI)
writer.SetInputData(prob_image)
writer.Write()

print(f"Saved output to {OUTPUT_VTI}")


<>:7: SyntaxWarning: invalid escape sequence '\I'
<>:7: SyntaxWarning: invalid escape sequence '\I'
C:\Users\telug\AppData\Local\Temp\ipykernel_25328\768221482.py:7: SyntaxWarning: invalid escape sequence '\I'
  INPUT_VTI = "data\Isabel_3D_gmm.vti"


3 3125000 3 3125000 3 3125000
3 50 3 50 3 50
Component 0 shape: (50, 250, 250)
Component 1 shape: (50, 250, 250)
Component 2 shape: (50, 250, 250)


Processing 8D GMM Cells:  14%|█▎        | 413631/3038049 [26:01<1:55:45, 377.88it/s]

#mutliprocessing gmm

In [ ]:
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from tqdm import tqdm
from itertools import product
import multiprocessing
import os
import glob
# === CONFIG ===
ISOLEVEL = 159.9798
N_SAMPLES = 10000
vti_dir = "data"  # Change to your actual path if needed
vti_files = glob.glob(os.path.join(vti_dir, "*.vti"))
for vti_file in vti_files:
    INPUT_VTI = os.path.basename(vti_file)
    OUTPUT_VTI  = f"isosurface_{INPUT_VTI }"
    INPUT_VTI = vti_file
    OUTPUT_VTI = os.path.join(vti_dir,OUTPUT_VTI )

    print(f"Processing: {INPUT_VTI} -> {OUTPUT_VTI}")
    # === Load VTI File ===
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(INPUT_VTI)
    reader.Update()
    image = reader.GetOutput()
    dims = image.GetDimensions()

    # === Load GMM Arrays (Means, Stds, Weights) ===
    means = [vtk_to_numpy(image.GetPointData().GetArray(f"GMM_Mean{i}")) for i in range(3)]
    stds = [vtk_to_numpy(image.GetPointData().GetArray(f"GMM_Std{i}")) for i in range(3)]
    weights = [vtk_to_numpy(image.GetPointData().GetArray(f"GMM_Weight{i}")) for i in range(3)]
    print(len(means), len(means[0]), len(stds), len(stds[0]), len(weights), len(weights[0]))

    # === Reshape Arrays to 3D Grids ===
    gmm_means_3d = [arr.reshape(dims[::-1], order='F') for arr in means]
    gmm_stds_3d = [arr.reshape(dims[::-1], order='F') for arr in stds]
    gmm_weights_3d = [arr.reshape(dims[::-1], order='F') for arr in weights]
    print(len(gmm_means_3d), len(gmm_means_3d[0]), len(gmm_stds_3d), len(gmm_stds_3d[0]), len(gmm_weights_3d), len(gmm_weights_3d[0]))
    for i, mean in enumerate(gmm_means_3d):
        print(f"Component {i} shape: {mean.shape}")

    # === Cell Vertex Indexing ===
    def get_cell_vertices(i, j, k):
        return [
            (i,   j,   k),
            (i+1, j,   k),
            (i+1, j+1, k),
            (i,   j+1, k),
            (i,   j,   k+1),
            (i+1, j,   k+1),
            (i+1, j+1, k+1),
            (i,   j+1, k+1),
        ]

    # === GMM Component Sampler ===
    def sample_8d_gmm(weights, means, covs, n_samples):
        comps = np.random.choice(3, size=n_samples, p=weights)
        samples = np.empty((n_samples, 8))
        for c in range(3):
            idx = np.where(comps == c)[0]
            if len(idx) > 0:
                samples[idx] = np.random.multivariate_normal(mean=means[c], cov=covs[c], size=len(idx))
        return samples

    # === Probability Estimator ===
    def sample_crossing_probability_joint_gmm(weights, means, covs, isovalue, n_samples):
        samples = sample_8d_gmm(weights, means, covs, n_samples)
        cross = np.any(samples < isovalue, axis=1) & np.any(samples > isovalue, axis=1)
        return np.mean(cross)

    # === Main Computation Function ===
    def compute_probability_at_cell(idx):
        i, j, k = idx
        verts = get_cell_vertices(i, j, k)

        means_list = []
        covs_list = []
        weights_raw = []

        for c in range(3):
            mu_c = [gmm_means_3d[c][x, y, z] for (x, y, z) in verts]
            std_c = [gmm_stds_3d[c][x, y, z] for (x, y, z) in verts]
            cov_c = np.diag(np.square(std_c))

            means_list.append(np.array(mu_c))
            covs_list.append(cov_c)

            w_c = [gmm_weights_3d[c][x, y, z] for (x, y, z) in verts]
            w_product = np.prod(w_c)
            weights_raw.append(w_product)

        weights_array = np.array(weights_raw)
        weights_array /= np.sum(weights_array)

        prob = sample_crossing_probability_joint_gmm(weights_array, means_list, covs_list, ISOLEVEL, N_SAMPLES)
        return (i, j, k, prob)

    # === Prepare Grid and Indices ===
    output_shape = tuple(d - 1 for d in dims[::-1])
    prob_grid = np.zeros(output_shape, dtype=np.float32)
    all_indices = list(product(range(output_shape[0]),
                              range(output_shape[1]),
                              range(output_shape[2])))

    # === Run Multiprocessing ===
    if __name__ == '__main__':
        with multiprocessing.Pool() as pool:
            for i, j, k, prob in pool.imap_unordered(compute_probability_at_cell, all_indices):
                prob_grid[i, j, k] = prob

        # === Convert to VTK ImageData ===
        prob_image = vtk.vtkImageData()
        prob_image.SetDimensions(prob_grid.shape)
        prob_image.SetSpacing(image.GetSpacing())
        prob_image.SetOrigin(image.GetOrigin())

        flat = prob_grid.ravel(order='F')
        vtk_array = numpy_to_vtk(flat, deep=True)
        vtk_array.SetName("crossing_probability")
        prob_image.GetPointData().AddArray(vtk_array)
        prob_image.GetPointData().SetActiveScalars("crossing_probability")

        # === Write Output ===
        writer = vtk.vtkXMLImageDataWriter()
        writer.SetFileName(OUTPUT_VTI)
        writer.SetInputData(prob_image)
        writer.Write()

        print(f"Saved output to {OUTPUT_VTI}")


Processing: /content/data/sklearn_teardrop_gmm.vti -> /content/data/isosurface_sklearn_teardrop_gmm.vti
3 262144 3 262144 3 262144
3 64 3 64 3 64
Component 0 shape: (64, 64, 64)
Component 1 shape: (64, 64, 64)
Component 2 shape: (64, 64, 64)


Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 131, in worker
    put((job, i, result))
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 377, in put
    self._writer.send_bytes(obj)
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  

EOFError: Ran out of input

  File "/usr/lib/python3.11/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 427, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 430, in _recv_bytes
    buf = self._recv(4)
          ^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 384, in _send
    n = write(self._handle, buf)
        ^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
